## 

# Conway's Game of Life
<p align="center">
<img src="GoL_table.png" alt="drawing" width="200" />
</p>
The universe of the Game of Life is an infinite two-dimensional orthogonal grid of square cells, each of which is in one of two possible states, live or dead. Every cell interacts with its eight neighbours, which are the cells that are directly horizontally, vertically, or diagonally adjacent. At each step in time, the following transitions occur:

1. **Underpopulation/Exposure**: Any live cell with fewer than two live neighbours dies.
2. **Overpopulation/Overcrowding**: Any live cell with more than three live neighbours dies.
3. **Stasis**: Any live cell with two or three live neighbours lives, unchanged, to the next generation.
4. **Reproduction**: Any dead cell with exactly three live neighbours will come to life.

The initial pattern constitutes the 'seed' of the system. The first generation is created by applying the above rules simultaneously to every cell in the seed — births and deaths happen simultaneously, and the discrete moment at which this happens is sometimes called a tick/generation. The rules continue to be applied repeatedly to create further generations.




## 2. Simple Implementations
Ignoring the complexities of loading patterns and display output, the basic algorithm for the GoL can be very simple. 

A naive approach to a simulating the GoL would be to go through every cell, check for any alive neighbours, and determine the state of the cell in the next generation. While this would work, it would be inefficient for larger grid sizes where only a fraction of livecells may be found. If we had an infinitely sized board, an iterative solution by scanning all cells wouldn't work. There are a few solutions around this which have implications to how the cells interact generation-to-generation, and for animation purposes. A few solutions are present:

1. Using a sparse matrix that can change size, expanding in any direction to contain all the live cells.
2. Using a set of live cells, where a live cell is represented by its (x,y) coordinate in the 1D board. 

If we have a collection of “live” cells stored as (x,y) pairs, a simple Python program to compute the next generation is shown:

In [1]:
import collections
'''
1.  By iteratively tallying coordinates surrounding the alive cell itself, we can determine the number of connections that the coordinate has to an alive cell. If there are three occurances of the coordinates counted, we know exactly that the coordinate will be alive in the next generation. If there are less, or more than three connections for the coordinate, we know its going to die of underpopulation, or overpopulation, respectively.
2.  Now that we have a set of coordinates with the number of connections (i.e., the occurance of cells surrounding an alive cell in the initial generation), we can filter out only for the alive cells in the next generation by assigning a Discrete Transform Function according to the table below. "Transform states" 2 & 3 only convert to a live cell in the next generation.

Original | New | Transform State
    0       0           0
    1       0           1
    0       1           2
    1       1           3
'''

def gameOfLifeInfinite(live,generations):
    for i in range(generations):   
        print(live)
        # count alive neighbouring cells except for the alive cell itself
        ctr = collections.Counter((I, J) for r, c in live for I in range(r-1, r+2) for J in range(c-1, c+2) if I != r or J != c) 
        # return a set of coordinates with the newest generation of live cells according to transform states 2 & 3
        live = {ij for ij in ctr if ctr[ij] == 3 or ctr[ij] == 2 and ij in live} 

Given the input provided in the sample input test case, we can see that the two live cells in the extreme bottom left of the universe dies from exposure. Note that Python3's numbers are not bound by CPU registry limitations (64-bit level numbers). Rather Python3's numbering is limited by system RAM limitations. Let's note this as a feature of Python, as we don't need to play around with "unsigned long long" types in C/C++.

In [2]:
live = sorted({(0, 1), (1, 2), (2, 0), (2, 1), (2, 2), (-2**64, -2**64), (-2**64+1, -2**64+1)})

generations = 3
gameOfLifeInfinite(live,generations)

[(-18446744073709551616, -18446744073709551616), (-18446744073709551615, -18446744073709551615), (0, 1), (1, 2), (2, 0), (2, 1), (2, 2)]
{(1, 2), (2, 1), (3, 1), (2, 2), (1, 0)}
{(1, 2), (3, 1), (2, 0), (2, 2), (3, 2)}


The previously given simple algorithm runs at ~10,000 generations per second on my Macbook M1 Pro, so it should be easily fast enough to animate a large 256 x 256 universe at 30 generations per second. If all we want is a pretty animation (and ignore loading and saving patterns), there’s likely to be no reason to do anything more complicated. But given we are handling large integers we have to assume that we may encounter large patterns in future generations, so lets try to entertain a faster implementation. 

## Advanced Implementations



**An Algorithm for Compressing Space & Time**
|<img src ="quadtree_steps.png" alt="drawing" width="200"/>|
|:--:| 
| *A representation of a 2-D [Bounding Volume Hierarchy](https://en.wikipedia.org/wiki/Bounding_volume_hierarchy), the [Quadtree](https://en.wikipedia.org/wiki/Quadtree)* |

For advanced implementations, we will focus on calculating subsequent generations, ignoring the impact on other operations such as displaying the pattern, and saving, or loading patterns. Since we are exploring an algorithmic solution, Python works as a perfect testbed without the worries of integer overflows. However, it would be important to point out some inherent benefits (i.e., to cell generation speed and memory management) in utilizing an alternative language like C/C++. Accelerating Conway's GoL using carefully written C++ code/intrinsics may yield higher performance by using Single Instruction Multiple Data (SIMD) instructions.  

For instance, if we were to represent a cell state in 4-bits as shown in the transform states above (i.e., in a unsigned long long ), we could possibly represent 16-cell states in 64-bit wide words (SSE). Naively speaking, this would allow us to pack 16-neighbor cell counts in a single 64-bit word representation, allowing us to calculate 16 resulting cells. Using explicit SIMD instructions to increase our register size to 256 bits (AVX2), or 512 bits (AVX-512) may potentially lead to doubling in performance. 

However...


|<img src = "Conways_game_of_life_breeder_animation.gif" width="400"/>|
|:--:| 
|*A Breeder*|

A "breeder" is a pattern that exhibits quadratic growth, by generating multiple copies of a secondary pattern, each of which then generates multiple copies of a tertiary pattern. "For this pattern, space increases at O(n^2) and computation time increases at O(n^3), so computing the pattern at generation 2n takes eight times as much time as computing the pattern at generation n." 

Having an input of this pattern would quickly drown out performance gains from modern SIMD, or parallelism implementations. 